In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt
from wordsegment import load, segment
load()

In [2]:
# read all files
category_df = pd.read_csv('categories.csv')
full_df = pd.read_csv('full_data.csv')
outfit_df = pd.read_csv('outfits.csv')
tag_df = pd.read_csv('tagged_product_attributes.csv')
review_df = pd.read_csv('womens_clothing_reviews.csv')

In [3]:
category_df

,value,category,type,quantifier
0,Androgynous,style,general,many
1,Athleisure,style,general,many
2,Boho,style,general,many
3,Business Casual,style,general,many
4,Casual,style,general,many
...,...,...,...,...
898,Snakeskin,material,subcategory_scarves,many
899,Suede,material,subcategory_scarves,many
900,Synthetic,material,subcategory_scarves,many
901,Textile,material,subcategory_scarves,many


In [4]:
full_df

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id
0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,514683,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an...","{""Needs Review""}",NaN
1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,526676,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...,"{""Needs Review""}",NaN
2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,4.001E+11,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection\nCase and cleaning cloth in...,"{""Needs Review""}",NaN
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,4.00012E+11,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,Canvas upper\nRound toe\nLace-up vamp\nSmartFO...,"{""Needs Review""}",NaN
4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,4.00011E+11,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection\nGradient lenses\nAdjustabl...,"{""Needs Review""}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48974,01DSNVXY8EJ9FQAJ3MPDMPASHD,Bonpoint,4.00091E+11,Baby's Hooded Jacket,NaN,JustKids/Baby024months/InfantGirls/Outerwear,2019-11-14 21:08:28.040417+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/bonpoint-babys...,Cozy double breasted jacket crafted from cotto...,"{""Needs Review""}",NaN
48975,01DSGYHA3RMCHENBJVQPBGXM97,Laura Mercier,4.00096E+11,Flawless Fusion Ultra-Longwear Foundation,"WHAT IT ISA 15-hour long wearing, water resist...",SaksBeautyPlace/ForHer/Color/Foundation/Liquid...,2019-11-12 23:17:47.761072+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/laura-mercier-...,"WHAT IT ISA 15-hour long wearing, water resist...","{""Needs Review""}",NaN
48976,01DSJT8H12CAFQQH07SQSQWJ8C,Splendid,4.001E+11,Baby Girl's 2-Piece Ruffle Sweatshirt & Stripe...,Ruffled-trim sweatshirt lends romance to this ...,"JustKids/Baby024months/InfantGirls/Tops,JustKi...",2019-11-13 16:41:34.491443+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/splendid-baby-...,"Crewneck\nLong sleeves\nRib-knit neck, cuffs a...","{""Needs Review""}",NaN
48977,01DSH2PF9J7QZ44D842B3GMCFN,Florence Eiseman,4.00012E+11,Little Girl's Plaid & Velvet Dress,Pretty plaid dress with velvet collar and velv...,"JustKids/Girls214/ToddlerGirls24/Dresses,JustK...",2019-11-13 00:30:31.212215+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/florence-eisem...,Peter Pan collar\nShort sleeves\nBack zipper\n...,"{""Needs Review""}",NaN


In [5]:
outfit_df

,outfit_id,product_color_id,outfit_item_type
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S83FARJW4QWRBXNS,shoe
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2V2Y8EN8YPY1E16EV,accessory1
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA23E5A8GBES8QJN3HF,top
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2C7BQN0X7MNNYXBHW,bottom
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2S83FARJW4QWRBXNS,shoe
...,...,...,...
5587,01E6MC52DNZE5AZ9MPZ2CYCXW9,01E4RW25YJBXGTTW54236ECJXQ,onepiece
5588,01E6MC52DPCTAAQVYNSKA7PER7,01E5ZS3RA3XV8JAN4R16HD1TA3,accessory1
5589,01E6MC52DPCTAAQVYNSKA7PER7,01E4RW25YJBXGTTW54236ECJXQ,onepiece
5590,01E6MC52DPCTAAQVYNSKA7PER7,01E2P0SJT1FXRYBGJCPF1P0XQ8,shoe


In [6]:
# have to do some cleaning/preprocessing
# since the attribute names and values are not completely identical 
# between the initial_tags and additional tags
tag_df

,product_id,product_color_id,attribute_name,attribute_value,file
0,01DVBTBPHR8WJTCVEN5AJRHF47,01DVBTBPJ41VVT00JJCG8TTZ2W,gender,Women,initial_tags
1,01DVA7QRXM928ZM0WWR7HFNTC1,01DVA7QRXXR9F0TWVE1HMC5ZQ3,Primary Color,Blacks,initial_tags
2,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
3,01E1JM43NQ3H17PB22EV3074NX,01E1JM5WFWWCCCH3JTTTCYQCEQ,style,Modern,initial_tags
4,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
...,...,...,...,...,...
119340,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,fit,relaxed,additional
119341,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,lengthblazerscoatsandjackets,midlengthathips,additional
119342,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,closureblazerscoatsandjackets,buttonedfront,additional
119343,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,occasion,work,additional


In [8]:
def seg(s):
    if not re.search(r'_', s):
        l = segment(s)
        if len(l) == 1:
            return l[0]
        else:
            return '_'.join(l)
    else:
        return s

tag_df['attribute_name_new'] = tag_df.attribute_name.apply(seg)
tag_df['attribute_value_new'] = tag_df.attribute_value.apply(seg)

In [9]:
sorted(tag_df.attribute_name_new.unique())

['additional_color',
 'belt_buckle_material',
 'belt_buckle_shape',
 'belt_closure',
 'belt_material',
 'belt_width',
 'calf_width',
 'category',
 'class_belts',
 'class_blazers_coats_and_jackets',
 'class_booties',
 'class_boots',
 'class_dress',
 'class_flats',
 'class_handbags',
 'class_jumpsuit_and_romper',
 'class_mules_and_slides',
 'class_pants_and_leggings',
 'class_pumps_and_heels',
 'class_sandals',
 'class_shorts',
 'class_skirts',
 'class_slippers',
 'class_sneakers_and_athletic',
 'class_sunglasses',
 'class_wedges',
 'closure_blazers_coats_and_jackets',
 'closure_handbag',
 'closure_one_piece',
 'closure_pants_and_leggings',
 'closure_shoe',
 'closure_shorts',
 'closure_skirts',
 'closure_sweater',
 'closure_top',
 'color',
 'dry_clean_only',
 'embellishment',
 'fit',
 'gender',
 'heel_height',
 'heel_shape',
 'leg_style',
 'leg_style_jeans',
 'length_blazers',
 'length_blazers_coats_and_jackets',
 'length_coats_and_jackets',
 'length_jeans',
 'length_one_piece',
 'length

In [10]:
sorted(tag_df.attribute_value_new.unique())

['100_cashmere',
 '100_cotton',
 '100linen',
 '100silk',
 '100wool',
 '14zip',
 '5pocketpantnondenim',
 'a_line',
 'abstract',
 'accessory',
 'acetate',
 'acid',
 'acrylic',
 'active',
 'active_track',
 'adjustable',
 'alpaca',
 'androgynous',
 'animal',
 'ankle',
 'ankle_strap',
 'anorak_raincoat_windbreaker',
 'asymmetrical',
 'athleisure',
 'aviator_square',
 'aviator_teardrop',
 'back_tie',
 'back_zip',
 'back_zipper',
 'backless',
 'backpacks',
 'baggy',
 'ballet',
 'band_collar',
 'beach_bags',
 'beaded',
 'beiges',
 'belt',
 'belt_bags_fannypack',
 'belted',
 'belts',
 'bermuda',
 'bike',
 'black',
 'blacks',
 'blazer',
 'blazer_dress',
 'blazers_coats_jackets',
 'block',
 'blouse',
 'blues',
 'boatneck',
 'body_con',
 'bodysuit',
 'boho',
 'bomber_varsity',
 'bootcut',
 'bootie',
 'booties',
 'boots',
 'bottom',
 'boyfriend',
 'brass',
 'browns',
 'bucket_bags',
 'buckle',
 'buckles',
 'burgundies',
 'business_casual',
 'business_casual_dress',
 'bustier',
 'button_down',
 'but

In [11]:
review_df

,clothing_id,age,title,review_text,rating,recommend,likes,division,department,class
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...
23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


# Decide which N – 2 categories your group would like to focus on. 

You are required to focus on the proprietary attributes – style and occasion. Beyond that, your group should pick N - 2 other groups to analyze.

Examples of categories:
- Embellishments
- Category
- Prints
- Material

# Join the tagged_product_attributes table with full_data and investigate the details, descriptions, and tags used for each category

your goal is to get a sense for the business logic and rules used in tagging a certain product category.

In [12]:
df = pd.merge(full_df, tag_df, on = 'product_id')
df['bc_product_id'] = df['bc_product_id'].astype('Int64')
df = df.astype(str)
df

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id,product_color_id,attribute_name,attribute_value,file,attribute_name_new,attribute_value_new
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,5529544,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,2020-04-15 21:59:56.695000+00:00,2020-04-17 15:44:57.785000+00:00,2020-04-17 15:44:57.785000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",[],5021,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend,additional,material_clothing,linen_blend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,5529544,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,2020-04-15 21:59:56.695000+00:00,2020-04-17 15:44:57.785000+00:00,2020-04-17 15:44:57.785000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",[],5021,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend,additional,material_clothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,5529544,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,2020-04-15 21:59:56.695000+00:00,2020-04-17 15:44:57.785000+00:00,2020-04-17 15:44:57.785000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",[],5021,01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern,additional,style,modern
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,5529544,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,2020-04-15 21:59:56.695000+00:00,2020-04-17 15:44:57.785000+00:00,2020-04-17 15:44:57.785000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",[],5021,01E5ZXP5JCREDC7WJVMWHK5Q40,style,businesscasual,additional,style,business_casual
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,5529544,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,2020-04-15 21:59:56.695000+00:00,2020-04-17 15:44:57.785000+00:00,2020-04-17 15:44:57.785000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",[],5021,01E5ZXP5JCREDC7WJVMWHK5Q40,style,classic,additional,style,classic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159008,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,4788992,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,2020-03-04 23:28:52.433000+00:00,2020-04-02 23:33:25.438000+00:00,2020-04-02 23:33:25.438000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",[],1818,01E2KY2T9NK7SA5C1RZN4DK64B,occasion,weekend,additional,occasion,weekend
159009,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,4788992,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,2020-03-04 23:28:52.433000+00:00,2020-04-02 23:33:25.438000+00:00,2020-04-02 23:33:25.438000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",[],1818,01E2KY2T9NK7SA5C1RZN4DK64B,sleevelength,shortsleeve,additional,sleeve_length,short_sleeve
159010,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,4788992,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,2020-03-04 23:28:52.433000+00:00,2020-04-02 23:33:25.438000+00:00,2020-04-02 23:33:25.438000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",[],1818,01E2KY2T9NK7SA5C1RZN4DK64B,sizing,regular,additional,sizing,regular
159011,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,4788992,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,2020-03-04 23:28:52.433000+00:00,2020-04-02 23:33:25.438

In [13]:
df.to_csv('joined.csv')

# Build a model that will takes as input:
- product description (if any)
- product name
- product details (if any)
- brand

# And outputs the predicted attributes of this product.

For example, if the category you are using is fit,

INPUT:
- description: Blush linen Button fastenings along front 100% linen; lining: 100% cotton Dry clean Designer color: Shell Imported
- brand: Zimmermann
- brand_category: Clothing / Jumpsuits / Full Length

The actual clothing’s product URL is here.

OUTPUT:
- predicted fit: RELAXED